In [1]:
import cv2
import numpy as np

In [81]:
confThreshold = 0.5  #Confidence threshold
nmsThreshold = 0.4   #Non-maximum suppression threshold
inpWidth = 224       #Width of network's input image
inpHeight = 224      #Height of network's input image

In [3]:
# Load names of classes
classesFile = "D:\Repository\darknet\cfg\coco.names"
classes = None
with open(classesFile, 'rt') as f:
    classes = f.read().rstrip('\n').split('\n')

# Give the configuration and weight files for the model and load the network using them.
modelConfiguration = "D:\Repository\darknet\cfg\yolov3.cfg"
modelWeights = "D:\Repository\darknet\yolov3.weights"

net = cv2.dnn.readNetFromDarknet(modelConfiguration, modelWeights)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

In [50]:
temp[0][0][100][5:][0]

0.0

In [56]:
cv2.dnn.NMSBoxes([[306, 189, 47, 76]], [0.68193567], confThreshold, nmsThreshold)

array([0])

In [83]:
def postprocess(frame, outs):
    fHeight = frame.shape[0]
    fWidth = frame.shape[0]
    
    classIds = []
    confidences = []
    boxes = []
    
    for out in outs:
        for detection in out:
            scores = detection[5:]
            classId = np.argmax(scores)
            confidence = scores[classId]
            if confidence > confThreshold:
                center_x = int(detection[0] * fWidth)
                center_y = int(detection[1] * fHeight)
                width = int(detection[2] * fWidth)
                height = int(detection[3] * fHeight)
                left = int(center_x - width/2)
                top = int(center_y - height/2)
                classIds.append(classId)
                confidences.append(float(confidence))
                boxes.append([left, top, width, height])
                
    indices = cv2.dnn.NMSBoxes(boxes, confidences, confThreshold, nmsThreshold)
    
    for i in indices:
        box = boxes[i]
        left = box[0]
        top = box[1]
        width = box[2]
        height = box[3]
        
        drawPred(classIds[i], confidences[i], left, top, left+width, top+height)
        

def drawPred(classId, conf, left, top, right, bottom):
    print(left, top, right, bottom)
    print('')
    # Draw a bounding box.
    cv2.rectangle(frame, (left, top), (right, bottom), (255, 178, 50), 3)
    
    label = '%.2f' % conf
        
    # Get the label for the class name and its confidence
    if classes:
        assert(classId < len(classes))
        label = '%s:%s' % (classes[classId], label)
        
    #Display the label at the top of the bounding box
    labelSize, baseLine = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
    top = max(top, labelSize[1])
    cv2.rectangle(frame, (left, top - round(1.5*labelSize[1])), (left + round(1.5*labelSize[0]), top + baseLine), (255, 255, 255), cv2.FILLED)
    
    
    
def getOutputsNames(net):
    # Get the names of all the layers in the network
    layersNames = net.getLayerNames()
    
    # Get the names of the output layers, i.e. the layers with unconnected outputs
    return [layersNames[i - 1] for i in net.getUnconnectedOutLayers()]

In [84]:
video = "traffic.mp4"
cap = cv2.VideoCapture(video)

temp = []
while(cap.isOpened()):
    ret, frame = cap.read()
    
    if ret:
        blob = cv2.dnn.blobFromImage(frame, 1/255, (inpWidth, inpHeight), [0,0,0], 1, crop = False)
        net.setInput(blob)
        outs = net.forward(getOutputsNames(net))
        temp.append(outs)
        postprocess(frame, outs)
        
        t, _ = net.getPerfProfile()
        label = 'Inference time: %.2f ms' % (t * 1000.0 / cv2.getTickFrequency())
        cv2.putText(frame, label, (0, 15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255))
        

        cv2.imshow('Frame', frame)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

  # Break the loop
    else: 
        break

# When everything done, release the video capture object
cap.release()

# Closes all the frames
cv2.destroyAllWindows()

200 81 224 108

155 80 184 115

226 147 251 190

313 184 348 269

207 22 224 38

126 277 175 366

226 167 254 203

154 82 183 116

202 82 224 109

123 279 174 376

207 22 225 39

224 169 254 206

313 181 348 270

223 168 255 209

200 83 223 112

155 81 185 116

208 24 225 41

118 295 166 383

199 91 222 118

222 170 253 214

155 81 185 117

114 293 167 392

207 24 226 43

198 90 223 121

112 301 167 404

155 81 185 117

222 170 252 216

207 27 225 44

199 92 225 125

155 80 185 117

221 168 251 220

208 28 226 46

105 313 162 415

314 194 349 275

197 93 224 126

221 169 251 221

154 79 185 116

209 27 229 46

100 318 161 427

314 195 349 276

332 292 380 413

196 98 223 130

207 31 228 50

155 80 185 115

222 171 250 222

102 325 160 435

314 194 349 276

196 100 224 132

218 182 252 238

154 80 185 115

207 31 227 52

317 201 354 276

196 100 223 134

155 80 185 115

220 188 251 242

209 35 226 53

318 202 356 276

196 101 224 136

219 184 251 247

155 81 184 115

318 203 355 278

20